# Density and public transportation

Sufficient population and job density are important for a public transportation system. Therefore plotting densities and public transportation systems on the same map can reveal opportuntities to build more public transportation lines (where density is high, but there are no existing lines) and to build more housing/jobs (where public transportation lines exist, but densities are low).

First, let's create a map of population density. In the United States, we can get this geo-tagged population data from the United States Census Bureau. dpd.modeling has a class Zones to store this data and a method to automatically pull this data from the United States Census Bureau.

First, we'll define the counties that we are interested in mapping and the latest year for US Census data.

States seems to be the best level to get data for. If we get data by county, there are lots of requests so it takes too long. However, data for the whole country is unnecessary. The government assigns a number to each state so California is 06.

Now we can get the census data for California. B01003_001E is the population in each census tract.

Next we compute density as population per area.

And we filter for the counties we listed above. This way we can change our filter below and not have to redownload all the data.

Now we download our public transportation systems so we can plot the lines.

And finally we plot everything. (Image omitted to reduce file size.)

Now, it would be helpful to do the same exercise with job densiites. However, the US Census Bureau does not provide this information ("Worker Population": "B08604_001E") at the tract level like they do for population. This leaves us with two options which both require large downloads.

1. We can download zip code worker populations.
2. Or we can download LODES data which includes origin-destination information.

We'll take a look at option 2 below. The LODES data is divided into three files: residential data, work data, and origin-destination data. There is also a cross-walk file that includes a translation from LODES GEOIDs to census tracts.

We can then combine the LODES data with our original output DataFrame (which includes the geometry) to add a job_density column.

And we can plot the job density like we ploted the population density above.

Last, we can evaluate the sum of population density and job density. This sum gives the best measure of the number of potential public transportation users in each census tract.

In [1]:
import us
import ipywidgets as widgets
from IPython.display import display

In [2]:
YEAR = "2017"

state = widgets.Select(
    options=list(map(lambda x: x.name, us.STATES)),
    description="State",
    value="California",
)
display(state)

Select(description='State', index=4, options=('Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Color…

In [3]:
from dpd.modeling import Zones

zones = Zones.from_uscensus(str(us.states.lookup(state.value).fips), YEAR)

In [4]:
zones["geometry"] = zones["geometry"].apply(lambda x: x.simplify(0.001))

In [5]:
zones.head(1)

,Name,Production,Attraction,state,county,tract,STATEFP,COUNTYFP,TRACTCE,NAME_y,...,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,ProductionAttractionSum,Production Density,Attraction Density,ProductionAttractionSum Density
GEOID,,,,,,,,,,,,,,,,,,,,,
6107001400,"Census Tract 14, Tulare County, California",5721,0,06,107,001400,06,107,001400,14,...,S,124522201,0,+36.3259261,-119.1202602,"POLYGON ((-119.22469 36.32643, -119.22029 36.3...",5721,45.943615,0.0,45.943615


In [6]:
# zones.explore(column="ProductionAttractionSum Density")

In [7]:
from tobler.util import h3fy
from tobler.area_weighted import area_interpolate
from pyproj import CRS

aea = CRS.from_string("North America Albers Equal Area Conic")
zones.to_crs(aea)
h3_zones = h3fy(zones, buffer=True)

dc_hex_interpolated = area_interpolate(
    source_df=zones,
    target_df=h3_zones,
    intensive_variables=["Production", "Attraction", "ALAND"],
)
dc_hex_interpolated.head()

/usr/local/lib/python3.11/site-packages/tobler/util/util.py:113: UserWarning: The source geodataframe is stored in a geographic CRS. Falling back to estimated UTM zone to generate desired buffer. If this produces unexpected results, reproject the input data prior to using this function
  warn(
/var/folders/r4/0h9hr9rd7gx0k7mb58t92vcm0000gn/T/ipykernel_97505/457227677.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dc_hex_interpolated = area_interpolate(


,Production,Attraction,ALAND,geometry
hex_id,,,,
862814a97ffffff,2579.124573,0.0,1.804784e+09,"POLYGON ((-121.32890 39.86695, -121.29644 39.8..."
8629a9bafffffff,9973.401407,0.0,2.306312e+08,"POLYGON ((-121.30783 36.39647, -121.27643 36.4..."
862814387ffffff,1833.000000,0.0,7.772392e+08,"POLYGON ((-121.23914 40.09377, -121.20658 40.1..."
862833c6fffffff,5852.307102,0.0,8.127251e+08,"POLYGON ((-122.13567 39.01085, -122.10384 39.0..."
8629a4747ffffff,7792.222200,0.0,8.804190e+06,"POLYGON ((-117.23038 33.11858, -117.19836 33.1..."


In [8]:
# Zones(dc_hex_interpolated).explore(column="ProductionAttractionSum Density")

In [9]:
h3_zones = h3fy(zones, resolution=8, buffer=True)

gdf = area_interpolate(
    source_df=zones,
    target_df=h3_zones,
    intensive_variables=["Production", "Attraction", "ALAND"],
)
gdf = Zones(gdf)
gdf.head()

/usr/local/lib/python3.11/site-packages/tobler/util/util.py:113: UserWarning: The source geodataframe is stored in a geographic CRS. Falling back to estimated UTM zone to generate desired buffer. If this produces unexpected results, reproject the input data prior to using this function
  warn(
/var/folders/r4/0h9hr9rd7gx0k7mb58t92vcm0000gn/T/ipykernel_97505/3272913931.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf = area_interpolate(


,Production,Attraction,ALAND,geometry,ProductionAttractionSum,Production Density,Attraction Density,ProductionAttractionSum Density
hex_id,,,,,,,,
8829a041d1fffff,2157,0,6.727593e+08,"POLYGON ((-116.73393 34.10418, -116.72929 34.1...",2157,3.206199,0.0,3.206199
882836d1e9fffff,9633,0,2.178794e+07,"POLYGON ((-120.54798 37.31374, -120.54341 37.3...",9633,442.125302,0.0,442.125302
8828307841fffff,2839,0,4.427160e+08,"POLYGON ((-121.86544 38.50321, -121.86090 38.5...",2839,6.412688,0.0,6.412688
8828155695fffff,4478,0,2.992359e+09,"POLYGON ((-122.06040 40.37067, -122.05579 40.3...",4478,1.496478,0.0,1.496478
8829a6ab33fffff,2693,0,2.757441e+09,"POLYGON ((-116.34856 33.39556, -116.34393 33.3...",2693,0.976630,0.0,0.976630


In [10]:
from lonboard import Map, HeatmapLayer, SolidPolygonLayer
from lonboard.colormap import apply_continuous_cmap
from palettable.matplotlib import Viridis_20

layer = SolidPolygonLayer.from_geopandas(gdf, opacity=0.2)
df = gdf["ProductionAttractionSum Density"]
normalized_df = (df - df.min()) / (df.max() - df.min())

layer.get_fill_color = apply_continuous_cmap(normalized_df, Viridis_20)

m = Map(layers=[layer])
# m

Map(layers=[SolidPolygonLayer(get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x13a6b8f40>
[
  [
    …

In [11]:
from dpd.driving.network import Network

query = """
[out:json][timeout:25];
(
  relation["network"="Metro Rail"];

);
out body;
>;
out skel qt;
"""

network = Network.from_osm_query(query)

100%|█████████████████████████████████████████████████████| 14/14 [00:45<00:00,  3.24s/it]


In [12]:
import folium

folium_map = folium.Map()
zones.explore(m=folium_map, column="ProductionAttractionSum Density")
for route in network.routes:
    network.routes[route].explore(m=folium_map)

# folium_map

In [13]:
import gtfs_kit

feed = gtfs_kit.read_feed(
    # "http://www.bart.gov/dev/schedules/google_transit.zip", dist_units="mi"
    "https://gtfs.sfmta.com/transitdata/google_transit.zip",
    dist_units="mi",
)

In [14]:
feed.routes

,route_id,agency_id,route_short_name,route_long_name,route_url,route_desc,route_type,route_color,route_text_color,route_sort_order
0,1,SFMTA,1,CALIFORNIA,http://www.sfmta.com/1,5am-12 midnight daily,3,005B95,FFFFFF,NaN
1,12,SFMTA,12,FOLSOM-PACIFIC,http://www.sfmta.com/12,6am-10pm daily,3,005B95,FFFFFF,NaN
2,14,SFMTA,14,MISSION,http://www.sfmta.com/14,24 hour service daily,3,005B95,FFFFFF,NaN
3,14R,SFMTA,14R,MISSION RAPID,http://www.sfmta.com/14R,5am-10pm daily,3,BF2B45,FFFFFF,NaN
4,15,SFMTA,15,BAYVIEW HUNTERS POINT EXPRESS,http://www.sfmta.com/15,Weekdays 5am-10pm Weekends 8am-10pm,3,005B95,FFFFFF,NaN
...,...,...,...,...,...,...,...,...,...,...
65,S,SFMTA,S,SHUTTLE,http://www.sfmta.com/S,Additional Weekday Service,0,FFD600,000000,NaN
66,T,SFMTA,T,THIRD,http://www.sfmta.com/T,Weekends 8 am-11:30 pm,0,BF2B45,FFFFFF,NaN
67,TBUS,SFMTA,TBUS,THIRD BUS,http://www.sfmta.com/TBUS,Weekdays 5am-6am Weekends 5am-8am,3,BF2B45,FFFFFF,NaN
68,P,SFMTA,P,POWELL CABLE CAR BUS,http://www.sfmta.com/P,6 am - 11:30 pm daily,3,B49A36,000000,NaN


In [15]:
from dpd.driving import Network

network = Network.from_gtfs(feed)

In [16]:
from dpd.modeling import TripDataFrame

In [17]:
od = TripDataFrame.from_lodes(us.states.lookup(state.value).abbr.lower(), YEAR)

/usr/local/lib/python3.11/site-packages/dpd/modeling/trip_dataframe.py:83: DtypeWarning: Columns (27,28,29,37) have mixed types. Specify dtype option on import or set low_memory=False.
  xwalk = pandas.read_csv(download_lodes_xwalk(st))


In [18]:
od.head()

w_geocode         h_geocode  S000  SA01  SA02  \
trct_w     trct_h                                                             
6001400100 6001400100  1440336024025368  1440336024024947    29     3     3   
           6001400200    60014237002025    60014002001020     1     0     0   
           6001400300   540126009009607   540126027022100     9     2     6   
           6001400400   300070455005125   300070255029072     6     1     4   
           6001400500   360084006006408   360084030010035     7     2     2   

                       SA03  SE01  SE02  SE03  SI01  SI02  SI03  createdate  
trct_w     trct_h                                                            
6001400100 6001400100    23     5     3    21     2     1    26   484826640  
           6001400200     1     0     1     0     0     0     1    20201110  
           6001400300     1     4     2     3     0     0     9   181809990  
           6001400400     1     2     3     1     0     0     6   101005550  
           6001400500     3     2     2     3     0     0     7   121206660

In [19]:
zones = zones.production_attraction_from_lodes(od)

In [20]:
h3_zones = h3fy(zones, resolution=8, buffer=True)

gdf = area_interpolate(
    source_df=zones,
    target_df=h3_zones,
    intensive_variables=["Production", "Attraction", "ALAND"],
)
gdf = Zones(gdf)
gdf.head()

/usr/local/lib/python3.11/site-packages/tobler/util/util.py:113: UserWarning: The source geodataframe is stored in a geographic CRS. Falling back to estimated UTM zone to generate desired buffer. If this produces unexpected results, reproject the input data prior to using this function
  warn(
/var/folders/r4/0h9hr9rd7gx0k7mb58t92vcm0000gn/T/ipykernel_97505/3272913931.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf = area_interpolate(


,Production,Attraction,ALAND,geometry,ProductionAttractionSum,Production Density,Attraction Density,ProductionAttractionSum Density
hex_id,,,,,,,,
8829a041d1fffff,1293,321,6.727593e+08,"POLYGON ((-116.73393 34.10418, -116.72929 34.1...",1614,1.921935,0.477139,2.399075
882836d1e9fffff,3537,730,2.178794e+07,"POLYGON ((-120.54798 37.31374, -120.54341 37.3...",4267,162.337506,33.504772,195.842278
8828307841fffff,1224,15295,4.427160e+08,"POLYGON ((-121.86544 38.50321, -121.86090 38.5...",16519,2.764752,34.548105,37.312857
8828155695fffff,1665,1019,2.992359e+09,"POLYGON ((-122.06040 40.37067, -122.05579 40.3...",2684,0.556417,0.340534,0.896951
8829a6ab33fffff,1153,1113,2.757441e+09,"POLYGON ((-116.34856 33.39556, -116.34393 33.3...",2266,0.418141,0.403635,0.821777


In [21]:
network.routes["1"]

,geometry,name
0,POINT (-122.39678 37.79547),Clay St & Drumm St
1,POINT (-122.39697 37.79544),nan
2,POINT (-122.39678 37.79547),Clay St & Drumm St
3,POINT (-122.39660 37.79459),nan
4,POINT (-122.39761 37.79446),Sacramento St & Davis St
...,...,...
142,POINT (-122.49238 37.78179),32nd Ave & Clement St
143,POINT (-122.49239 37.78168),nan
144,POINT (-122.49225 37.77992),32nd Ave & Geary Blvd
145,POINT (-122.49224 37.77975),nan


In [22]:
from lonboard import Map, PathLayer, ScatterplotLayer
from lonboard.colormap import apply_continuous_cmap
from palettable.matplotlib import Viridis_20

layer = SolidPolygonLayer.from_geopandas(gdf, opacity=0.2)
df = gdf["ProductionAttractionSum Density"]
normalized_df = (df - df.min()) / (df.max() - df.min())

layer.get_fill_color = apply_continuous_cmap(normalized_df, Viridis_20)
layers = [layer]
for route in network.routes:
    layers.append(
        ScatterplotLayer.from_geopandas(network.routes[route], radius_min_pixels=1)
    )
m = Map(layers=layers)
# m

Map(layers=[SolidPolygonLayer(get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x106706620>
[
  [
    …

In [23]:
from astropy import units
from dpd.driving import Route

route = Route.from_osm_relation(relation=2351006)

In [24]:
from dpd.modeling import DistanceDataFrame

zones.to_crs("North_America_Albers_Equal_Area_Conic", inplace=True)
points = zones.polygons_to_points()
stops = route.stops.to_crs("North_America_Albers_Equal_Area_Conic")
distance_dataframe = DistanceDataFrame.from_origins_destinations(
    points.geometry, stops.geometry, method="distance"
)

In [25]:
points

,GEOID,geometry,Production,Attraction,ProductionAttractionSum
0,6107001400,POINT (-1951289.090 -192556.732),62.636364,47.333333,109.969697
1,6107001400,POINT (-1951289.090 -190839.969),62.636364,47.333333,109.969697
2,6107001400,POINT (-1951289.090 -189123.206),62.636364,47.333333,109.969697
3,6107001400,POINT (-1949171.655 -201140.549),62.636364,47.333333,109.969697
4,6107001400,POINT (-1949171.655 -194273.496),62.636364,47.333333,109.969697
...,...,...,...,...,...
348186,6111007508,POINT (-1980472.367 -438664.127),43.897436,19.948718,63.846154
348187,6111007508,POINT (-1980472.367 -438371.923),43.897436,19.948718,63.846154
348188,6111007508,POINT (-1980329.127 -438956.331),43.897436,19.948718,63.846154
348189,6111007508,POINT (-1980329.127 -438664.127),43.897436,19.948718,63.846154


In [26]:
distance_dataframe.columns = stops.name
distance_dataframe

name,Atlantic,East LA Civic Center,Maravilla,Indiana,Soto,Mariachi Plaza,Pico/Aliso,Little Tokyo/Arts District,Historic Broadway,Grand Avenue Arts/Bunker Hill,...,Expo/La Brea,La Cienega/Jefferson,Culver City,Palms,Westwood/Rancho Park,Expo/Sepulveda,Expo/Bundy,26th Street/Bergamot,17th Street/SMC,Downtown Santa Monica
0,282902.051899,282695.850548,282511.305433,281732.988112,280190.244854,279591.825116,279369.213180,278922.898918,278333.468571,277943.085451,...,278782.945046,278227.718885,277685.092967,277189.174674,275921.773539,275876.927681,275917.890569,276021.366201,276364.230748,277197.265609
1,284612.408972,284406.535040,284222.283077,283444.881660,281902.676907,281304.541733,281082.152628,280636.215691,280047.005542,279656.744315,...,280499.161901,279944.130511,279401.655703,278905.839090,277638.513195,277593.687132,277634.640276,277738.055003,278080.843192,278913.766470
2,286322.842725,286117.292351,285933.330079,285156.833761,283615.161330,283017.307387,282795.138451,282349.574335,281760.581794,281370.441029,...,282215.385403,281660.546424,281118.220889,280622.504717,279355.253143,279310.446633,279351.390150,279454.744721,279797.457489,280630.270543
3,274170.559252,273967.548746,273785.974208,273017.732356,271481.590694,270886.894492,270667.324294,270226.452242,269640.392154,269251.942383,...,270154.966846,269609.677339,269077.139298,268590.682462,267334.940213,267296.827579,267351.279696,267466.750815,267818.633865,268661.482810
4,281015.268816,280813.509376,280633.052099,279868.283005,278334.177001,277740.547952,277521.807630,277082.334001,276497.088023,276109.087679,...,277020.901526,276476.156257,275943.980845,275457.701020,274201.973191,274163.772094,274217.846961,274332.771353,274684.120688,275526.276180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348186,64480.421075,63855.803145,63285.790127,61302.903097,59395.930403,58502.107190,57941.593997,56921.598570,56105.344127,55608.182658,...,48735.156992,47342.808902,45972.160749,44720.308851,42747.855597,42104.161858,40950.452693,40022.690683,39555.262502,39438.980417
348187,64642.807010,64019.083362,63449.913843,61469.352644,59560.595843,58666.540592,58106.680220,57087.670761,56270.957200,55773.334604,...,48931.018633,47541.190539,46173.153773,44923.784993,42952.136928,42311.263476,41163.638869,40241.472040,39779.042598,39669.419077
348188,64199.467616,63574.249289,63003.671960,61019.243312,59113.481700,58219.822943,57658.876480,56638.230842,55822.291744,55325.444567,...,48433.418848,47039.672239,45667.602262,44414.425290,42441.562778,41796.400688,40639.652138,39709.264356,39239.639557,39120.677325
348189,64361.235785,63736.911169,63167.176988,61185.069047,59277.491352,58383.588931,57823.293245,56803.628366,55987.217056,55489.899301,...,48628.739983,47237.519420,45868.068362,44617.382730,42645.309157,42002.986980,40852.370171,39927.624668,39463.045809,39350.810177


times = [5, 10, 15]
data = []
for column in distance_dataframe.columns:
    row = []
    for time in times:
        # 1.35 meters/second and 60 seconds per minute
        row.append(
            points[(distance_dataframe / 1.35 < time * 60)[column]][
                "ProductionAttractionSum"
            ].sum()
        )
    data.append(row)

from pandas import DataFrame

DataFrame(data=data, index=distance_dataframe.columns, columns=times).plot(kind="bar")

from matplotlib import pyplot as plt

fig, ax = plt.subplots()
(distance_dataframe / 1.35).hist(
    weights=points["ProductionAttractionSum"],
    range=(0, 900),
    bins=30,
    cumulative=True,
    sharey=True,
    ax=ax,
)
ax.set_ylabel("Population (cumulative)")
ax.set_xlabel("Time (seconds)")